In [1]:
from clique_benchmark import Trials

In [2]:
# run this cell to load the job results for all trials that are waiting pending job results
trials = Trials()
await trials.load_results()

In [7]:
print(trials.get(graph_id=74)[0].graph.g)
print(trials.get(graph_id=74)[0].clique_size)

011011
2


In [4]:
print(sorted(trials.get(graph_id=74)[0].counts.items(), key=lambda t: t[1], reverse=True))
print(sorted(trials.get(graph_id=74)[0].simulation_counts.items(), key=lambda t: t[1], reverse=True))

[('0101', 715), ('0111', 710), ('1101', 670), ('1100', 665), ('1000', 657), ('0000', 654), ('0100', 646), ('1010', 639), ('1111', 631), ('0110', 630), ('1110', 618), ('0011', 596), ('0010', 573), ('1011', 563), ('1001', 526), ('0001', 507)]
[('0101', 2050), ('1001', 1916), ('1100', 1892), ('1010', 1856), ('1101', 1852), ('0000', 52), ('0100', 48), ('1111', 47), ('0010', 44), ('0011', 43), ('1000', 40), ('1011', 40), ('1110', 37), ('0001', 31), ('0111', 28), ('0110', 24)]
